In [79]:
import pandas as pd
import numpy as np

ah_data=pd.read_csv('C:/Users/admin/Desktop/EeD/Python/Project/AmesHousingData.csv')
ah_data['age'] = ah_data['YrSold'].apply(lambda x: x - min(ah_data['YrSold']))
        
#dropId
ah_data = ah_data.drop(['Id', 'SaleType'], axis = 1)

#to display all cloumns
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 1000)
ah_data.sample(5)


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,age
772,80,RL,94.0,7819,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,1Fam,SLvl,6,5,1976,1976,Gable,CompShg,Plywood,Plywood,None,0.0,TA,TA,CBlock,TA,TA,Av,ALQ,422,BLQ,127,480,1029,GasA,TA,Y,SBrkr,1117,0,0,1117,1,0,1,0,3,1,TA,6,Typ,1,TA,Detchd,1976.0,Unf,2,672,TA,TA,Y,144,0,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Abnorml,107000,4
208,60,RL,NaN,14364,Pave,NaN,IR1,Low,AllPub,Inside,Mod,SawyerW,Norm,Norm,1Fam,2Story,7,5,1988,1989,Gable,CompShg,Plywood,Plywood,BrkFace,128.0,Gd,TA,CBlock,Gd,TA,Gd,GLQ,1065,Unf,0,92,1157,GasA,Ex,Y,SBrkr,1180,882,0,2062,1,0,2,1,3,1,TA,7,Typ,1,Gd,Attchd,1988.0,Fin,2,454,TA,TA,Y,60,55,0,0,154,0,NaN,NaN,NaN,0,4,2007,WD,Normal,277000,1
1255,50,RM,52.0,6240,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,BrkSide,Norm,Norm,1Fam,1.5Fin,6,6,1931,1950,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,TA,TA,BrkTil,TA,Fa,No,LwQ,425,Unf,0,459,884,GasA,TA,Y,FuseA,959,408,0,1367,0,0,1,0,3,1,TA,6,Typ,1,Gd,Detchd,1978.0,Unf,1,560,TA,TA,Y,0,0,0,0,120,0,NaN,NaN,NaN,0,11,2007,WD,Normal,127500,1
1384,50,RL,60.0,9060,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,1Fam,1.5Fin,6,5,1939,1950,Gable,CompShg,WdShing,Wd Shng,None,0.0,TA,TA,BrkTil,TA,TA,Mn,Rec,204,Unf,0,356,560,GasA,TA,Y,SBrkr,698,560,0,1258,0,0,1,0,2,1,TA,6,Typ,0,NaN,Detchd,1939.0,Unf,1,280,TA,TA,P,0,0,0,0,0,0,NaN,MnPrv,NaN,0,10,2009,WD,Normal,105000,3
338,20,RL,91.0,14145,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,NWAmes,Norm,Norm,1Fam,1Story,7,7,1984,1998,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,Gd,TA,CBlock,Gd,TA,Mn,ALQ,213,Unf,0,995,1208,GasA,Ex,Y,SBrkr,1621,0,0,1621,1,0,2,0,3,1,Gd,8,Typ,0,NaN,Attchd,1984.0,RFn,2,440,TA,TA,Y,108,45,0,0,0,0,NaN,NaN,Shed,400,5,2006,WD,Normal,202500,0


In [80]:
print(ah_data.shape)

(1460, 81)


In [81]:
X = ah_data.drop(['SalePrice'],axis = 1)
y = ah_data.SalePrice
print(X.shape)

(1460, 80)


In [82]:
from sklearn.model_selection import train_test_split
train_X,val_X,train_y,val_y = train_test_split(X,y,test_size = 0.2,random_state = 42)

for item in [train_X,val_X,train_y,val_y]:
    print(item.shape)

(1168, 80)
(292, 80)
(1168,)
(292,)


In [83]:
from sklearn.base import TransformerMixin
from sklearn.base import BaseEstimator

class CustomImputer(BaseEstimator, TransformerMixin):
    def __init__(self, strategy='mean',filler='NA'):
       self.strategy = strategy
       self.fill = filler
    
    def fit(self, X, y=None):
       if self.strategy in ['mean','median']:
           if not all(X.dtypes == np.number):
               raise ValueError('dtypes mismatch np.number dtype is \
                                 required for '+ self.strategy)
       if self.strategy == 'mean':
           self.fill = X.mean()
       elif self.strategy == 'median':
           self.fill = X.median()
       elif self.strategy == 'mode':
           self.fill = X.mode().iloc[0]
       elif self.strategy == 'fill':
           if type(self.fill) is list and type(X) is pd.DataFrame:
               self.fill = dict([(cname, v) for cname,v in zip(X.columns, self.fill)])
       return self

    def transform(self, X, y=None):
       return X.fillna(self.fill)

In [84]:
#ah_data = DataFrameImputer().fit_transform(ah_data)
#ah_data.sample(5)   
train_X = CustomImputer(strategy = 'mode').fit_transform(train_X)
val_X = CustomImputer(strategy = 'mode').fit_transform(val_X)
train_y = CustomImputer(strategy = 'mode').fit_transform(train_y)
val_y = CustomImputer(strategy = 'mode').fit_transform(val_y)

In [85]:
#seggregating columns as per data type
integer_columns = train_X.select_dtypes(include=['int64']).columns.tolist()
#integer_columns = integer_columns[1:65]
char_text_columns = train_X.select_dtypes(include=['object']).columns.tolist()
float_columns = train_X.select_dtypes(include = ['float64']).columns.tolist()

#we are picking out only integer and float columns for now
numbers_columns = float_columns + integer_columns
all_columns = float_columns + integer_columns + char_text_columns
print(all_columns)

['LotFrontage', 'MasVnrArea', 'GarageYrBlt', 'MSSubClass', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'age', 'MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'Gar

In [86]:
train_X = train_X[all_columns]
val_X=val_X[all_columns]
train_X = pd.get_dummies(train_X)
val_X = pd.get_dummies(val_X)
train_X.head(5)

,LotFrontage,MasVnrArea,GarageYrBlt,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,age,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,Alley_Grvl,Alley_Pave,LotShape_IR1,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_Bnk,LandContour_HLS,LandContour_Low,LandContour_Lvl,Utilities_AllPub,Utilities_NoSeWa,LotConfig_Corner,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LandSlope_Gtl,LandSlope_Mod,LandSlope_Sev,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Condition1_Artery,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Artery,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,Condition2_RRAe,Condition2_RRAn,Condition2_RRNn,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Fin,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Flat,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,RoofMatl_ClyTile,RoofMatl_CompShg,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,Exterior1st_AsbShng,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Stone,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_AsbShng,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Other,Exterior2nd_Plywood,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,MasVnrType_BrkCmn,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,ExterQual_Ex,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,ExterCond_Ex,ExterCond_Fa,ExterCond_Gd,ExterCond_Po,ExterCond_TA,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,BsmtQual_Ex,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_TA,BsmtCond_Fa,BsmtCond_Gd,BsmtCond_Po,BsmtCond_TA,BsmtExposure_Av,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_No,BsmtFinType1_ALQ,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_Rec,BsmtFinType1_Unf,BsmtFinType2_ALQ,BsmtFinType2_BLQ,BsmtFinType2_GLQ,BsmtFinType2_LwQ,BsmtFinType2_Rec,BsmtFinType2_Unf,Heating_Floor,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,HeatingQC_Ex,HeatingQC_Fa,HeatingQC_Gd,HeatingQC_Po,HeatingQC_TA,CentralAir_N,CentralAir_Y,Electrical_FuseA,Electrical_FuseF,Electrical_FuseP,Electrical_SBrkr,KitchenQual_Ex,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,Functional_Maj1,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sev,Functional_Typ,FireplaceQu_Ex,FireplaceQu_Fa,FireplaceQu_Gd,FireplaceQu_Po,FireplaceQu_TA,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageFinish_Fin,GarageFinish_RFn,GarageFinish_Unf,GarageQual_Ex,GarageQual_Fa,GarageQual_Gd,GarageQual_Po,GarageQual_TA,GarageCond_Ex,Garage

In [87]:
from sklearn.ensemble import ExtraTreesRegressor
model = ExtraTreesRegressor()
model.fit(train_X, train_y)
feature_scores = model.feature_importances_
print(model.feature_importances_)


[  2.91891426e-03   2.27696146e-03   2.06583954e-03   1.71166712e-03
   5.80539405e-03   2.67543701e-01   2.49434049e-03   2.16629349e-03
   4.98880652e-03   8.84639451e-03   1.33639905e-03   1.31740965e-03
   1.12011997e-02   1.10757505e-02   1.23121125e-02   5.80357626e-05
   5.28690049e-02   6.55246404e-03   5.11464425e-04   2.48302183e-02
   1.50315780e-03   6.59152863e-03   1.25473040e-03   7.10054454e-03
   8.28782682e-03   1.55858415e-01   1.24658623e-02   1.85551926e-03
   2.95202303e-03   1.48236900e-03   1.26294326e-03   2.92734402e-03
   4.05539240e-03   7.46869490e-05   1.97694149e-03   1.36017640e-03
   1.03876264e-03   2.96268643e-05   3.95901042e-04   1.04151149e-05
   2.03757032e-03   2.96287013e-03   0.00000000e+00   0.00000000e+00
   3.27209732e-04   2.05080746e-04   5.25413681e-04   3.28234583e-03
   4.16752696e-04   1.70272725e-03   1.89153595e-03   6.63621943e-04
   2.89423592e-04   1.09364466e-03   6.57471278e-06   0.00000000e+00
   6.29394657e-04   1.00616997e-03

In [88]:
x_cn = train_X.columns
d = {'columnnames':x_cn, 'scores':model.feature_importances_}
temp_ahdata = pd.DataFrame(data = d)
selected_features = temp_ahdata.sort_values("scores").tail(30)
selected_features = selected_features['columnnames']
#type(selected_features) is pandas.core.series.Series
#hence, converting to list
selected_features = selected_features.tolist()
print(len(selected_features))
print(selected_features)

30
['BsmtExposure_Gd', 'LotShape_IR2', 'Exterior1st_HdBoard', 'BldgType_1Fam', 'BsmtQual_Gd', 'PoolArea', 'GarageType_Detchd', 'BsmtFinType1_GLQ', 'YearRemodAdd', 'KitchenQual_TA', 'SaleCondition_Abnorml', 'LotArea', 'CentralAir_N', 'BsmtFullBath', 'BedroomAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'BsmtFinSF1', '1stFlrSF', 'TotalBsmtSF', '2ndFlrSF', 'GarageArea', 'Neighborhood_NoRidge', 'GarageFinish_Unf', 'FullBath', 'BsmtQual_Ex', 'GrLivArea', 'ExterQual_TA', 'GarageCars', 'OverallQual']


In [89]:
train_X = train_X[selected_features]
val_X = val_X[selected_features]